In [1]:
#from kaggle_datasets import KaggleDatasets
#GCS_DS_PATH = KaggleDatasets().get_gcs_path()

In [ ]:
!pip install -U efficientnet
import efficientnet.keras as efn

     |████████████████████████████████| 50 kB 2.8 MB/s            


In [ ]:
pip install matplotlib2tikz

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import datetime
from itertools import permutations
from tqdm import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import re
import tensorflow as tf
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.metrics import classification_report
from keras.callbacks import LearningRateScheduler, ModelCheckpoint

from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE
from kaggle_datasets import KaggleDatasets


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

In [ ]:
!ls /kaggle/input/
#!ls /kaggle/working/

#!gsutil ls $GCS_PATH
GCS_PATH = KaggleDatasets().get_gcs_path('hopenet')
#print(GCS_PATH)

In [ ]:
models = []
histories = []

EPOCHS = 50
im = 512   # choose the image size 160, 224, 331, 512
IMAGE_SIZE = [im, im]
TARGET_SIZE = [im, im]

FLOWERS_DATASETS = { # available image sizes
    160: GCS_PATH + '/tfrecords-jpeg-160x160/train/*.tfrec',
    224: GCS_PATH + '/tfrecords-jpeg-224x224/train/*.tfrec',
    331: GCS_PATH + '/tfrecords-jpeg-331x331/train/*.tfrec',
    512: GCS_PATH + '/*.tfrec',
    #512: GCS_PATH + '/tfrecords-jpeg-512x512/train/*.tfrec',
}
CLASSES = ['cocci', 'healthy', 'salmo'] # do not change, maps to the labels in the data (folder names)
assert IMAGE_SIZE[0] == IMAGE_SIZE[1], "only square images are supported"
assert IMAGE_SIZE[0] in FLOWERS_DATASETS, "this image size is not supported"

IMAGE_SIZE[0]



# Learning rate schedule for TPU, GPU and CPU.
# Using an LR ramp up because fine-tuning a pre-trained model.
# Starting with a high LR would break the pre-trained weights.


In [ ]:
BATCH_SIZE = 2 * strategy.num_replicas_in_sync # this is 8 on TPU v3-8, it is 1 on CPU and
LR_START = 0.00001
LR_MAX = 0.000012 * strategy.num_replicas_in_sync
LR_MIN = 0.00001
LR_RAMPUP_EPOCHS = 10
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def lrfn(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr
    
lr_callback = tf.keras.callbacks.LearningRateScheduler(lrfn, verbose=True)

rng = [i for i in range(EPOCHS)]
y = [lrfn(x) for x in rng]
plt.plot(rng, y)
print("Learning rate schedule: {:.3g} to {:.3g} to {:.3g}".format(y[0], max(y), y[-1]))
BATCH_SIZE

In [ ]:
def dataset_to_numpy_util(dataset, N):
    dataset = dataset.unbatch().batch(N)
    for images, labels in dataset:
        numpy_images = images.numpy()
        numpy_labels = labels.numpy()
        break;  
    return numpy_images, numpy_labels

def title_from_label_and_target(label, correct_label):
    label = np.argmax(label, axis=-1)  # one-hot to class number
    correct_label = np.argmax(correct_label, axis=-1) # one-hot to class number
    correct = (label == correct_label)
    return "{} [{}{}{}]".format(CLASSES[label], str(correct), ', shoud be ' if not correct else '',
                                CLASSES[correct_label] if not correct else ''), correct

def display_one_flower(image, title, subplot, red=False):
    plt.subplot(subplot)
    plt.axis('off')
    plt.imshow(image)
    plt.title(title, fontsize=16, color='red' if red else 'black')
    return subplot+1


def display_9_images_from_dataset(dataset):
    subplot=331
    plt.figure(figsize=(13,13))
    images, labels = dataset_to_numpy_util(dataset, 9)
    for i, image in enumerate(images):
        title = CLASSES[np.argmax(labels[i], axis=-1)]
        subplot = display_one_flower(image, title, subplot)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()  

    
def display_9_images_with_predictions(images, predictions, labels):
    subplot=331
    plt.figure(figsize=(13,13))
    for i, image in enumerate(images):
        title, correct = title_from_label_and_target(predictions[i], labels[i])
        subplot = display_one_flower(image, title, subplot, not correct)
        if i >= 8:
            break;
              
    plt.tight_layout()
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.show()
    
def display_confusion_matrix(cmat, score, precision, recall):
    plt.figure(figsize=(15,15))
    ax = plt.gca()
    ax.matshow(cmat, cmap='Reds')
    ax.set_xticks(range(len(CLASSES)))
    ax.set_xticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_xticklabels(), rotation=45, ha="left", rotation_mode="anchor")
    ax.set_yticks(range(len(CLASSES)))
    ax.set_yticklabels(CLASSES, fontdict={'fontsize': 7})
    plt.setp(ax.get_yticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    titlestring = ""
    if score is not None:
        titlestring += 'f1 = {:.3f} '.format(score)
    if precision is not None:
        titlestring += '\nprecision = {:.3f} '.format(precision)
    if recall is not None:
        titlestring += '\nrecall = {:.3f} '.format(recall)
    if len(titlestring) > 0:
        ax.text(101, 1, titlestring, fontdict={'fontsize': 18, 'horizontalalignment':'right', 'verticalalignment':'top', 'color':'#804040'})
    plt.show()
    

def display_training_curves(training, validation, title, subplot):
    if subplot%10==1: # set up the subplots on the first call
        plt.subplots(figsize=(10,10), facecolor='#F0F0F0')
        plt.tight_layout()
    ax = plt.subplot(subplot)
    ax.set_facecolor('#F8F8F8')
    ax.plot(training)
    ax.plot(validation)
    ax.set_title('model '+ title)
    ax.set_ylabel(title)
    #ax.set_ylim(0.28,1.05)
    ax.set_xlabel('epoch')
    ax.legend(['train', 'valid.'])

In [ ]:
def count_data_items(filenames):
    # the number of data items is written in the name of the .tfrec files, i.e. flowers00-230.tfrec = 230 data items
    n = [int(re.compile(r"-([0-9]*)\.").search(filename).group(1)) for filename in filenames]
    return np.sum(n)



gcs_pattern = FLOWERS_DATASETS[IMAGE_SIZE[0]]
#validation_split = 0.15
valid_split = 0.2
test_split = 0.2
validation_split = valid_split + test_split
filenames = tf.io.gfile.glob(gcs_pattern)
split = len(filenames) - int(len(filenames) * validation_split)
TRAINING_FILENAMES = filenames[:split]

split = len(filenames) - int(len(filenames) * validation_split)
split1 = len(filenames) - int(len(filenames) * test_split)

VALIDATION_FILENAMES = filenames[split:split1]
TEST_FILENAMES = filenames[split1:]



TRAIN_STEPS = count_data_items(TRAINING_FILENAMES) // BATCH_SIZE
print("TRAINING IMAGES: ", count_data_items(TRAINING_FILENAMES), ", STEPS PER EPOCH: ", TRAIN_STEPS)
print("VALIDATION IMAGES: ", count_data_items(VALIDATION_FILENAMES))
print("TEST IMAGES: ", count_data_items(TEST_FILENAMES))

#NUM_TRAINING_IMAGES = count_data_items(TRAINING_FILENAMES)
NUM_VALIDATION_IMAGES = count_data_items(VALIDATION_FILENAMES)

def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string),  # tf.string = bytestring (not text string)
        "class": tf.io.FixedLenFeature([], tf.int64),   # shape [] means scalar
        
        # additional (not very useful) fields to demonstrate TFRecord writing/reading of different types of data
        "label":         tf.io.FixedLenFeature([], tf.string),  # one bytestring
        "size":          tf.io.FixedLenFeature([2], tf.int64),  # two integers
        "one_hot_class": tf.io.VarLenFeature(tf.float32)        # a certain number of floats
    }
    # decode the TFRecord
    example = tf.io.parse_single_example(example, features)
    
    # FixedLenFeature fields are now ready to use: exmple['size']
    # VarLenFeature fields require additional sparse_to_dense decoding
    
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    
    #image = tf.reshape(image, [*TARGET_SIZE, 3])
    
    #class_num = example['class']
    class_num = tf.cast(example['class'], tf.int32)

    
    label  = example['label']
    
    height = example['size'][0]
    width  = example['size'][1]
    one_hot_class = tf.sparse.to_dense(example['one_hot_class'])
    one_hot_class = tf.reshape(one_hot_class, [3])   # change whenever you changed the classes
    return image, one_hot_class



    #image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range
    #class_label = tf.cast(example['class'], tf.int32)
    #one_hot_class = tf.sparse.to_dense(example['one_hot_class'])
    #one_hot_class = tf.reshape(one_hot_class, [5])
    #return image, one_hot_class
    

def force_image_sizes(dataset, image_size):
    # explicit size needed for TPU
    reshape_images = lambda image, label: (tf.reshape(image, [*image_size, 3]), label)
    dataset = dataset.map(reshape_images, num_parallel_calls=AUTO)
    return dataset

def load_dataset(filenames):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_tfrecord, num_parallel_calls=AUTO)
    dataset = force_image_sizes(dataset, IMAGE_SIZE)
    return dataset

def data_augment(image, one_hot_class):
    # data augmentation. Thanks to the dataset.prefetch(AUTO) statement in the next function (below),
    # this happens essentially for free on TPU. Data pipeline code is executed on the "CPU" part
    # of the TPU while the TPU itself is computing gradients.
    #tf.image.resize_with_pad(image, target_height=IMAGE_SIZE[1], target_width=IMAGE_SIZE[1], method=tf.image.ResizeMethod.BILINEAR,antialias=False)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.resize_with_crop_or_pad(image, IMAGE_SIZE[1]+10, IMAGE_SIZE[1]+10) # Add 6 pixels of padding
    image = tf.image.random_crop(image, size=[IMAGE_SIZE[1], IMAGE_SIZE[1], 3]) # Random crop back to 28x28
    #%%%
    #image = tf.image.rotate(image,0.785)
    #image = tf.image.random_brightness(image, max_delta=0.5)
    #%%%
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_saturation(image, 0, 2)
    
    return image, one_hot_class  

def get_training_dataset():
    dataset = load_dataset(TRAINING_FILENAMES)
    dataset = dataset.map(data_augment, num_parallel_calls=AUTO)
    dataset = dataset.repeat()
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset(ordered=False):
    dataset = load_dataset(VALIDATION_FILENAMES)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

    #dataset = load_dataset(VALIDATION_FILENAMES, labeled=True, ordered=ordered)
    #dataset = dataset.batch(BATCH_SIZE)
    #dataset = dataset.cache()
    #dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    #return dataset
    
def get_test_dataset(ordered=False):
    dataset = load_dataset(TEST_FILENAMES)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset


In [ ]:
training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()
test_dataset = get_test_dataset()

In [ ]:
from keras import metrics
def create_model(num):
    if num == 1:
        #pretrained_model = tf.keras.applications.MobileNetV2(input_shape=[*IMAGE_SIZE, 3], include_top=False)
        #pretrained_model = tf.keras.applications.Xception(input_shape=[*IMAGE_SIZE, 3], include_top=False)
        pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
        
        #pretrained_model = tf.keras.applications.NASNetMobile(input_shape=[*IMAGE_SIZE, 3], include_top=True)
        #pretrained_model = tf.keras.applications.InceptionResNetV2(input_shape=[*IMAGE_SIZE, 3], include_top=False, weights="imagenet")
    elif num==2:
        #pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
        pretrained_model = efn.EfficientNetB0(input_shape = (512, 512, 3), include_top = False, weights = 'imagenet')
    else:
        #pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
        pretrained_model = tf.keras.applications.Xception(input_shape=[*IMAGE_SIZE, 3], include_top=False)
        #pretrained_model = tf.keras.applications.InceptionResNetV2(input_shape=[*IMAGE_SIZE, 3], include_top=False, weights="imagenet")
            
    #pretrained_model = tf.keras.applications.Xception(input_shape=[*IMAGE_SIZE, 3], include_top=False)
    #pretrained_model = tf.keras.applications.VGG16(weights='imagenet', include_top=False ,input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    #pretrained_model = tf.keras.applications.MobileNet(weights='imagenet', include_top=False, input_shape=[*IMAGE_SIZE, 3])
    # EfficientNet can be loaded through efficientnet.tfkeras library (https://github.com/qubvel/efficientnet)
    #pretrained_model = efficientnet.tfkeras.EfficientNetB0(weights='imagenet', include_top=False)
    
    pretrained_model.trainable = True

    model = tf.keras.Sequential([
        pretrained_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        #tf.keras.layers.Flatten(),
        #sigmoid
        #tf.keras.layers.Dense(1, activation='sigmoid')
        tf.keras.layers.Dense(3, activation='softmax')
        
        #tf.keras.layers.Dense(1)
    ])
    

   
    model.compile(
        optimizer='adam',
        #loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
        loss = 'categorical_crossentropy',
        #loss = tf.keras.losses.BinaryCrossentropy(from_logits=True),
        #metrics=[tf.keras.metrics.BinaryAccuracy()]
        #metrics = ['sparse_categorical_accuracy']
        metrics = ['accuracy']
    )

    return model

In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.optimizers import SGD
from keras.optimizers import Adam 
def define_model2():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(512, 512, 3)))
    model.add(MaxPooling2D((2, 2)))
    # futa
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    #futa
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    #model.add(Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    #model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    model.add(MaxPooling2D((2, 2)))
    #model.add(Conv2D(512, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
    #model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    #model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
    #model.add(Dense(2048, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(1024, activation='relu', kernel_initializer='he_uniform'))
    #model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
    model.add(Dense(3, activation='softmax'))
    # compile model
    #opt = SGD(lr=0.0001, momentum=0.9)
    opt = Adam()
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
with strategy.scope():
    #model = create_model(0)
    #model_1 = create_model(1)
    model_2 = create_model(2)
    #model_3 = define_model2()
    
#model.summary()
#model_1.summary()

#models.append(model)
#models.append(model_1)

In [ ]:
#output_dir = '/temp'

In [ ]:
#callbacks = [lr_callback, ModelCheckpoint(str(output_dir) + "/weights0.{epoch:03d}-{val_loss:.3f}-{val_accuracy:.3f}.hdf5",
#                             monitor="val_loss",
#                             verbose=1,
#                             save_best_only=True,
#                             mode="min")]

#date_1  = datetime.datetime.today()
#history = model.fit(training_dataset, validation_data=validation_dataset,
#                    steps_per_epoch=TRAIN_STEPS, epochs=EPOCHS, callbacks=callbacks)
#date_2 =  datetime.datetime.today()
#histories.append(history)

#final_accuracy = history.history["val_accuracy"][-5:]
#print("FINAL ACCURACY MEAN-5: ", np.mean(final_accuracy))

In [ ]:
#date_1  = datetime.datetime.today()
#history = model_1.fit(training_dataset, validation_data=validation_dataset,
#                    steps_per_epoch=TRAIN_STEPS, epochs=EPOCHS, callbacks=[lr_callback])
#date_2 =  datetime.datetime.today()
#histories.append(history)

#final_accuracy = history.history["val_accuracy"][-5:]
#print("FINAL ACCURACY MEAN-5: ", np.mean(final_accuracy))

In [ ]:
date_1  = datetime.datetime.today()
history = model_2.fit(training_dataset, validation_data=validation_dataset,
                    steps_per_epoch=TRAIN_STEPS, epochs= 15, callbacks=[lr_callback])
date_2 =  datetime.datetime.today()
histories.append(history)

final_accuracy = history.history["val_accuracy"][-5:]
print("FINAL ACCURACY MEAN-5: ", np.mean(final_accuracy))

In [ ]:
#from sklearn.externals import joblib
#import joblib


In [ ]:
#joblib_file = "efficient_net.pkl"   
#joblib.dump(model_2, joblib_file)

In [ ]:
#model_2.save("efficient_net.h5")

In [ ]:
model_2.save("efficient_net_withouth5")

In [ ]:
import shutil
shutil.make_archive('OUTPUT_NAME', 'zip', 'efficient_net_withouth5')

In [ ]:


callbacks = [lr_callback,
             ModelCheckpoint(str(output_dir) + "/weights.{epoch:03d}-{val_loss:.3f}-{val_accuracy:.3f}.hdf5",
                             monitor="val_loss",
                             verbose=1,
                             save_best_only=True,
                             mode="min")]
history = model_3.fit(training_dataset, validation_data=validation_dataset,
                    steps_per_epoch=TRAIN_STEPS, epochs= 100, callbacks=callbacks)

In [ ]:
time_delta = (date_2 - date_1)
total_seconds = time_delta.total_seconds()
minutes = total_seconds/60

print(minutes)

In [ ]:
df = pd.DataFrame(list(zip(history.history['accuracy'], history.history['val_accuracy'], history.history['loss'], history.history['val_loss'])), 
               columns =['Accuracy', 'val_accuracy', 'loss', 'val_loss']) 

In [ ]:
df.to_csv ('metricxnet224.csv', index = False, header=True)

In [ ]:
#history.history['accuracy']

In [ ]:
display_training_curves(history.history['accuracy'][1:], history.history['val_accuracy'][1:], 'accuracy', 211)
display_training_curves(history.history['loss'][1:], history.history['val_loss'][1:], 'loss', 212)

In [ ]:
display_training_curves(history.history['accuracy'][40:], history.history['val_accuracy'][40:], 'accuracy', 211)
display_training_curves(history.history['loss'][40:], history.history['val_loss'][40:], 'loss', 212)

In [ ]:
# a couple of images to test predictions too
some_flowers, some_labels = dataset_to_numpy_util(validation_dataset, count_data_items(VALIDATION_FILENAMES))

In [ ]:
# randomize the input so that you can execute multiple times to change results
permutation = np.random.permutation(NUM_VALIDATION_IMAGES)
some_flowers, some_labels = (some_flowers[permutation], some_labels[permutation])
date_1  = datetime.datetime.today()
predictions = model.predict(some_flowers, batch_size=16)

date_2  = datetime.datetime.today()
evaluations = model.evaluate(some_flowers, some_labels, batch_size=16)
  
print(np.array(CLASSES)[np.argmax(predictions, axis=-1)].tolist())
print('[val_loss, val_acc]', evaluations)

display_9_images_with_predictions(some_flowers, predictions, some_labels)


from sklearn.metrics import log_loss
some_flowers, some_labels = dataset_to_numpy_util(validation_dataset, NUM_VALIDATION_IMAGES)
predictions = model.predict(some_flowers, batch_size=16)
#evaluations = model.evaluate(some_flowers, some_labels, batch_size=16)
#clf_probs = clf.predict_proba(X_test)
score = log_loss(some_labels, predictions)
print(score)

In [ ]:
time_delta = (date_2 - date_1)
total_seconds = time_delta.total_seconds()
minutes = total_seconds/(60*NUM_VALIDATION_IMAGES)

print(minutes)

In [ ]:
minutes*60

In [ ]:
some_flowers, some_labels = dataset_to_numpy_util(test_dataset, count_data_items(TEST_FILENAMES)) 

In [ ]:
# randomize the input so that you can execute multiple times to change results
permutation = np.random.permutation(count_data_items(TEST_FILENAMES))
some_flowers, some_labels = (some_flowers[permutation], some_labels[permutation])
#print(type([np.argmax(some_labels, axis=-1)]))

predictions = model.predict(some_flowers, batch_size=16)
evaluations = model.evaluate(some_flowers, some_labels, batch_size=16)
  
print(np.array(CLASSES)[np.argmax(predictions, axis=-1)].tolist())
print('[val_loss, val_acc]', evaluations)

display_9_images_with_predictions(some_flowers, predictions, some_labels)


from sklearn.metrics import log_loss
some_flowers, some_labels = dataset_to_numpy_util(test_dataset, count_data_items(TEST_FILENAMES))
predictions = model.predict(some_flowers, batch_size=16)
#evaluations = model.evaluate(some_flowers, some_labels, batch_size=16)
#clf_probs = clf.predict_proba(X_test)
score = log_loss(some_labels, predictions)
results = np.array(CLASSES)[np.argmax(predictions, axis=-1)].tolist()
resultscon = np.argmax(predictions, axis=-1)
#print(classification_report(some_labels, predictions))
#print(score)
results1 = np.array(CLASSES)[np.argmax(some_labels, axis=-1)].tolist()
results1con = np.argmax(some_labels, axis=-1)

print(classification_report(results1, results, labels=['cocci', 'healthy', 'salmo']))

In [ ]:
from itertools import permutations
from tqdm import tqdm
from sklearn.metrics import accuracy_score
def generate_dim_ratios(n):
    switcher = {
        1: 1e-01,
        2: 1e-02,
        3: (1e-02)/2,
        4: (1e-02)/4,
        5: (1e-02)/8,
        6: (1e-02)/16,
        7: (1e-02)/32,
        8: (1e-02)/64,
        9: (1e-02)/128,
        10: (1e-02)/256,
        11: (1e-02)/512,
        12: (1e-02)/1024,
        }
    return switcher.get(n,"Invalid space")
#SUBMOD05:---------------------------------------------------------------------
def generate_weights_vector(ranks_prec_degree):
    #Generating initial weights distributions
    default_ratio = generate_dim_ratios(ranks_prec_degree)
    weights_vector = []
    weights_vector.append(0.0)
    for i in range (0,int(1/default_ratio) ):
        rational_wts = default_ratio + (default_ratio * i)
        weights_vector.append(round(rational_wts,12))    
    return default_ratio, weights_vector
#SUBMOD06:---------------------------------------------------------------------


#SUBMOD09:---------------------------------------------------------------------
def create_MatrixA_clrsweigths(weights_vector,ci):
    'maxrnk1_Perm_CLRSWGHTS_mxn'
    clrs_weights_matrix = []
    base_weight_vector = permutations(weights_vector,len(ci))#Permute clrswts
    for i in list (base_weight_vector):
        clrs_weights_matrix.append(i)
    clrs_weights_matrix =\
        pd.DataFrame(clrs_weights_matrix, columns=['dt','rf','nn','gb'])
    # Taking only the rows whose ranking totals to 1 
    MAT_A_CLRSWGHTS_mxn = clrs_weights_matrix\
        [(clrs_weights_matrix['dt'] + clrs_weights_matrix['rf'] + \
        clrs_weights_matrix['nn'] + clrs_weights_matrix['gb']) == 1] 
    return MAT_A_CLRSWGHTS_mxn # returns DataFrame
#SUBMOD10:---------------------------------------------------------------------
def gen_classifiers_initials(names):
    ci = []
    for c_i in range(0,len(names)):
        namep1,namep2 = names[c_i].split(' ',1)
        clrs_nmi = namep1[0]+namep2[0]
        ci.append(clrs_nmi.strip())
    return ci
#SUBMOD13:---------------------------------------------------------------------
    

In [ ]:
# randomize the input so that you can execute multiple times to change results
permutation = np.random.permutation(count_data_items(TEST_FILENAMES))
some_flowers, some_labels = (some_flowers[permutation], some_labels[permutation])

predictions = model.predict(some_flowers, batch_size=16)
np.savetxt("predictions.csv", predictions, delimiter=",")
predictions1 = model_1.predict(some_flowers, batch_size=16)
np.savetxt("predictions1.csv", predictions1, delimiter=",")
predictions2 = model_2.predict(some_flowers, batch_size=16)
np.savetxt("predictions2.csv", predictions2, delimiter=",")
predictions3 = model_3.predict(some_flowers, batch_size=16)
np.savetxt("predictions3.csv", predictions3, delimiter=",")
np.savetxt("some_labels.csv", some_labels, delimiter=",")



clrs_names = ["Decision Tree", "Random Forest", "Neural Net", "Gradient Boost"]
default_ratio, weights_vector = generate_weights_vector(1)
ci = gen_classifiers_initials(clrs_names)
# # #Matrix A --  permuted weights for available classifiers
MAT_A_CLRSWGHTS_mxn = create_MatrixA_clrsweigths(weights_vector,ci)
# print(MAT_A_CLRSWGHTS_mxn)
dt = predictions
rf = predictions1
nn = predictions2
gb = predictions3


In [ ]:
linker_dict = {'dt':dt,'rf':rf,'nn':nn,'gb':gb}
ci_of_probpred = ['dt','rf','nn','gb']
ci_of_probpred_perm = permutations(ci_of_probpred)
ci_of_probpred_perm_list = list(ci_of_probpred_perm)
k = list(permutations(ci_of_probpred))
for i in tqdm(range(len(ci_of_probpred_perm_list)), desc = "Creating grid search space"):
    ci_of_probpred_perm_list[i] = list(ci_of_probpred_perm_list[i])
    for j in range(len(ci_of_probpred_perm_list[i])):
       ci_of_probpred_perm_list[i][j] = linker_dict[ci_of_probpred_perm_list[i][j]]
MAT_B_CLRSPROBAPRED_nxm = ci_of_probpred_perm_list

In [ ]:
MAT_A_CLRSWGHTS_mxn = np.asarray(MAT_A_CLRSWGHTS_mxn)
MAT_A_1stexp = np.expand_dims(MAT_A_CLRSWGHTS_mxn,axis = 0)
MAT_A_2ndexp = np.expand_dims(MAT_A_1stexp,axis = 0)

MAT_A_2ndexp_rs = np.reshape(MAT_A_2ndexp,(MAT_A_CLRSWGHTS_mxn.shape[0],MAT_A_CLRSWGHTS_mxn.shape[1],1,1))

max_score, i_val, j_val = 0,0,0
for i in tqdm(range(MAT_A_2ndexp_rs.shape[0]), desc = "Optimizing Ensemble"):
    out1 = MAT_A_2ndexp_rs[i] * MAT_B_CLRSPROBAPRED_nxm 
    out2 = out1.sum(axis = 1)
    for j in range(out2.shape[0]):
        p = np.argmax(out2[j],axis = 1)
        # print(out2[j].shape)
        # if maximum < np.sum(out2[j])
        multi_en_probpred = p

        _DATA= [np.argmax(some_labels, axis=-1)]

        score = accuracy_score(multi_en_probpred,_DATA[0])        
        if score > max_score:
            print(score)
            max_score = score
            i_val = i
            j_val = j
            
            clrs_comb = list(k[j_val])
            print(clrs_comb)
            wts_comb = MAT_A_CLRSWGHTS_mxn[i_val]
            print(wts_comb)

#print(clrs_comb)
ensemb = ""

In [ ]:
df = pd.DataFrame(list(zip(results1, results)), columns =['YTest', 'YPredict']) 
df.to_csv ('con'+str(im)+'.csv', index = False, header=True)

In [ ]:
df = pd.DataFrame(list(zip(results1con, resultscon)), columns =['YTest', 'YPredict']) 
df.to_csv ('con'+str(im)+'label.csv', index = False, header=True)

In [ ]:
cmat = confusion_matrix(results1, results, labels=['cocci', 'healthy', 'salmo'])
score = f1_score(results1, results, labels=['cocci', 'healthy', 'salmo'], average='macro')
precision = precision_score(results1, results, labels=['cocci', 'healthy', 'salmo'], average='macro')
recall = recall_score(results1, results, labels=['cocci', 'healthy', 'salmo'], average='macro')
cmat = (cmat.T / cmat.sum(axis=1)).T # normalized
display_confusion_matrix(cmat, score, precision, recall)
print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))

In [ ]:
print(classification_report(results1, results, labels=['cocci', 'healthy', 'salmo']))

In [ ]:
model.save('hopenet.h5')

In [ ]:
model.save_weights('hopenet1.h5')

In [ ]:
#cmdataset = get_validation_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and labels, order matters.
#images_ds = cmdataset.map(lambda image, label: image)
#labels_ds = cmdataset.map(lambda image, label: label).unbatch()
#cm_correct_labels = next(iter(labels_ds.batch(NUM_VALIDATION_IMAGES))).numpy() # get everything as one batch
#cm_probabilities = model.predict(images_ds)
#cm_predictions = np.argmax(cm_probabilities, axis=-1)
#print("Correct   labels: ", cm_correct_labels.shape, cm_correct_labels)
#print("Predicted labels: ", cm_predictions.shape, cm_predictions)

In [ ]:
#cmat = confusion_matrix(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)))
#score = f1_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
#precision = precision_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
#recall = recall_score(cm_correct_labels, cm_predictions, labels=range(len(CLASSES)), average='macro')
#cmat = (cmat.T / cmat.sum(axis=1)).T # normalized
#display_confusion_matrix(cmat, score, precision, recall)
#print('f1 score: {:.3f}, precision: {:.3f}, recall: {:.3f}'.format(score, precision, recall))
